In [37]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle

In [38]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mathe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
print(stopwords.words('portuguese'))
stopwords_list = stopwords.words('portuguese')

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [40]:
df = pd.read_csv('noticias_pre_processadas.csv')

In [41]:
df.head()

,Texto,Categoria
0,O Podemos decidiu expulsar o deputado federal ...,true
1,"Bolsonaro é um liberal completo, diz president...",true
2,Ministro do STF libera Andrea Neves de prisão ...,true
3,"Apesar da abundância, cresce preocupação com p...",true
4,"Por que Harvard e MIT levarão Dilma, Moro e Su...",true


In [42]:
#criar função de steming

porter_stemmer = PorterStemmer() #busca a palavra raiz

def stemming_tokenizer(df):
    words = re.sub(r"[^A-Za-z]", " ", df).lower().split()
    words = [porter_stemmer.stem(word) for word in words if word not in stopwords_list]
    words = ' '.join(words)
    return words

In [43]:
df['features'] = df['Texto'].apply(stemming_tokenizer)

In [44]:
print(df['features'])

0       podemo decidiu expulsar deputado feder carlo g...
1       bolsonaro liber completo diz president psl par...
2       ministro stf libera andrea neve pri domiciliar...
3       apesar abund ncia cresc preocupa preserva font...
4       harvard mit levar dilma moro suplici eua megac...
                              ...                        
7194    ft julgamento populista usou dinheiro p blico ...
7195    nota fiscal emitida petrobra mostra gasolina v...
7196    est estancando sangria lava jato bem debaixo n...
7197    rj cidad fing possu pomba gira escapar blitz l...
7198    autor best seller cita express es indicam voc ...
Name: features, Length: 7199, dtype: object


In [45]:
#separar em features e labels
X = df['features'].values
Y = df['Categoria'].values

In [46]:
#Converter de texto para número
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
# Salvar o vetorizador em um arquivo
with open('vetorizador.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

In [47]:
X = vectorizer.transform(X)

In [48]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 42)

In [49]:
#Treinando o modelo
lr = LogisticRegression()
lr.fit(X_train, Y_train)

LogisticRegression()

In [64]:
#previsão
pred = lr.predict(X_test)

acc = accuracy_score(Y_test, pred)

f'Acurácia: {acc * 100:.2f}'

'Acurácia: 95.21'

In [51]:
#salvar modelo

with open('modelo_regressao_logistica.pkl', 'wb') as file:
    pickle.dump(lr, file)

In [67]:
from sklearn.metrics import classification_report

# Fazer previsões nos dados de teste
predicoes_teste = lr.predict(X_test)

# Avaliar as previsões usando um relatório de classificação
relatorio_classificacao = classification_report(Y_test, predicoes_teste)

print(relatorio_classificacao)

              precision    recall  f1-score   support

        fake       0.94      0.96      0.95       720
        true       0.96      0.94      0.95       720

    accuracy                           0.95      1440
   macro avg       0.95      0.95      0.95      1440
weighted avg       0.95      0.95      0.95      1440

